In [1]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten, Input
from keras.optimizers import RMSprop
from keras.utils.np_utils import to_categorical
from PIL import Image
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score
from keras import backend as K

Using TensorFlow backend.


In [ ]:
#tuning convolution kernel size and compare
#only use training set to do training and validation

In [2]:
#training label
labels = np.load('labels.npy')
labels.shape

(33402, 62)

In [3]:
#img as train data
size = labels.shape[0]
numsample = 5
folder = 'train/croppedsampled/'
images = []

for k in range(numsample):
    for i in range(size):
        im = Image.open(folder+str(i+1)+'_'+str(k)+'.png')
        images.append(np.asarray(im))
        
images = np.array(images)
images.shape

(167010, 54, 54, 3)

In [4]:
#replicate labels to cover all sample
labels = np.tile(labels,(5,1))
labels.shape

(167010, 62)

In [5]:
#split training set and validation set
rs = ShuffleSplit(n_splits=1, test_size=.1, random_state=0)
train_index, test_index = next(rs.split(images)) #just 1
test_images = images[test_index]
test_labels = labels[test_index]
images = images[train_index]
labels = labels[train_index]

In [6]:
#initial model (same as initial model in other program)
a = Input(shape=(54, 54, 3))
#conv layer
c1 = Conv2D(64, 3,3, activation='relu', border_mode='same')(a)
c1p = MaxPooling2D(pool_size=(2, 2)) (c1)
c1d = Dropout(0.1)(c1p)

c2 = Conv2D(128, 3,3, activation='relu', border_mode='same')(c1d)
c2p = MaxPooling2D(pool_size=(2, 2)) (c2)
c2d = Dropout(0.1)(c2p)

c3 = Conv2D(192, 3,3, activation='relu', border_mode='same')(c2d)
c3p = MaxPooling2D(pool_size=(2, 2)) (c3)
c3d = Dropout(0.1)(c3p)


c4 = Conv2D(256, 3,3, activation='relu', border_mode='same')(c3d)
c4p = MaxPooling2D(pool_size=(2, 2)) (c4)
c4d = Dropout(0.1)(c4p)


c5 = Conv2D(256, 3,3, activation='relu', border_mode='same')(c4d)
c5p = MaxPooling2D(pool_size=(2, 2)) (c5)
c5d = Dropout(0.1)(c5p)

#linear layer
flat = Flatten()(c5d)
d1 = Dense(128, activation='relu')(flat)
d1d = Dropout(0.1)(d1)

#output
o_length = Dense(7, activation='softmax')(d1d)
o_digit1 = Dense(11, activation='softmax')(d1d)
o_digit2 = Dense(11, activation='softmax')(d1d)
o_digit3 = Dense(11, activation='softmax')(d1d)
o_digit4 = Dense(11, activation='softmax')(d1d)
o_digit5 = Dense(11, activation='softmax')(d1d)
model = Model(a, [o_length,o_digit1,o_digit2,o_digit3,o_digit4,o_digit5])

rms = RMSprop(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=rms)


In [8]:
#training
result = model.fit(images, [labels[:,:7], labels[:,7:18], labels[:,18:29], labels[:,29:40], labels[:,40:51], labels[:,51:62]], 32, 20, validation_split=0.1, shuffle=True)

Train on 135278 samples, validate on 15031 samples
Epoch 1/20
135278/135278 [==============================] - 321s - loss: 5.5602 - dense_2_loss: 0.6068 - dense_3_loss: 1.7730 - dense_4_loss: 1.9770 - dense_5_loss: 0.9910 - dense_6_loss: 0.1923 - dense_7_loss: 0.0202 - val_loss: 3.7077 - val_dense_2_loss: 0.2931 - val_dense_3_loss: 1.1414 - val_dense_4_loss: 1.3668 - val_dense_5_loss: 0.7493 - val_dense_6_loss: 0.1556 - val_dense_7_loss: 0.0015
Epoch 2/20
135278/135278 [==============================] - 377s - loss: 3.2991 - dense_2_loss: 0.2683 - dense_3_loss: 0.9985 - dense_4_loss: 1.1987 - dense_5_loss: 0.6838 - dense_6_loss: 0.1463 - dense_7_loss: 0.0036 - val_loss: 2.4492 - val_dense_2_loss: 0.2165 - val_dense_3_loss: 0.7095 - val_dense_4_loss: 0.8433 - val_dense_5_loss: 0.5446 - val_dense_6_loss: 0.1336 - val_dense_7_loss: 0.0017- ETA: 19s - loss: 3.3261 - dense_2_loss: 0.2698 - dense_3_loss: 1.0082 - dense_4_loss: 1.2103 - dense_5_loss: 0.6883 - dense_6_loss: 0.1463 - dens - ET

In [9]:
#validation
score = model.predict(test_images)

score = np.concatenate(score, axis=1)
#round score to get actual prediction instead of probability
roundedScore = np.zeros(score.shape, dtype="int32")
#choose threshold
roundedScore[score > 0.5] = 1

#rounded acc (exactly match)
accuracy_score(test_labels, roundedScore)

0.79109035387102566

In [10]:
#conv kernel size = 5 (instead of 3 in initial model) 
a = Input(shape=(54, 54, 3))
#conv layer
c1 = Conv2D(64, 5,5, activation='relu', border_mode='same')(a)
c1p = MaxPooling2D(pool_size=(2, 2)) (c1)
c1d = Dropout(0.1)(c1p)

c2 = Conv2D(128, 5,5, activation='relu', border_mode='same')(c1d)
c2p = MaxPooling2D(pool_size=(2, 2)) (c2)
c2d = Dropout(0.1)(c2p)

c3 = Conv2D(192, 5,5, activation='relu', border_mode='same')(c2d)
c3p = MaxPooling2D(pool_size=(2, 2)) (c3)
c3d = Dropout(0.1)(c3p)


c4 = Conv2D(256, 5,5, activation='relu', border_mode='same')(c3d)
c4p = MaxPooling2D(pool_size=(2, 2)) (c4)
c4d = Dropout(0.1)(c4p)


c5 = Conv2D(256, 5,5, activation='relu', border_mode='same')(c4d)
c5p = MaxPooling2D(pool_size=(2, 2)) (c5)
c5d = Dropout(0.1)(c5p)

#linear layer
flat = Flatten()(c5d)
d1 = Dense(128, activation='relu')(flat)
d1d = Dropout(0.1)(d1)

#output
o_length = Dense(7, activation='softmax')(d1d)
o_digit1 = Dense(11, activation='softmax')(d1d)
o_digit2 = Dense(11, activation='softmax')(d1d)
o_digit3 = Dense(11, activation='softmax')(d1d)
o_digit4 = Dense(11, activation='softmax')(d1d)
o_digit5 = Dense(11, activation='softmax')(d1d)
model = Model(a, [o_length,o_digit1,o_digit2,o_digit3,o_digit4,o_digit5])

rms = RMSprop(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=rms)


In [12]:
#training
result = model.fit(images, [labels[:,:7], labels[:,7:18], labels[:,18:29], labels[:,29:40], labels[:,40:51], labels[:,51:62]], 32, 20, validation_split=0.1, shuffle=True)

Train on 135278 samples, validate on 15031 samples
Epoch 1/20
135278/135278 [==============================] - 415s - loss: 4.3098 - dense_9_loss: 0.4311 - dense_10_loss: 1.3354 - dense_11_loss: 1.5196 - dense_12_loss: 0.8199 - dense_13_loss: 0.1774 - dense_14_loss: 0.0264 - val_loss: 2.2920 - val_dense_9_loss: 0.1923 - val_dense_10_loss: 0.6798 - val_dense_11_loss: 0.7972 - val_dense_12_loss: 0.5004 - val_dense_13_loss: 0.1207 - val_dense_14_loss: 0.0015
Epoch 2/20
135278/135278 [==============================] - 413s - loss: 2.1429 - dense_9_loss: 0.1865 - dense_10_loss: 0.6279 - dense_11_loss: 0.7243 - dense_12_loss: 0.4729 - dense_13_loss: 0.1276 - dense_14_loss: 0.0038 - val_loss: 1.5260 - val_dense_9_loss: 0.1416 - val_dense_10_loss: 0.4424 - val_dense_11_loss: 0.5046 - val_dense_12_loss: 0.3328 - val_dense_13_loss: 0.1027 - val_dense_14_loss: 0.0019
Epoch 3/20
135278/135278 [==============================] - 413s - loss: 1.5773 - dense_9_loss: 0.1391 - dense_10_loss: 0.4644 - de

In [13]:
#validation
score = model.predict(test_images)

score = np.concatenate(score, axis=1)
#round score to get actual prediction instead of probability
roundedScore = np.zeros(score.shape, dtype="int32")
#choose threshold
roundedScore[score > 0.5] = 1

#rounded acc (exactly match)
accuracy_score(test_labels, roundedScore)

0.80977186994790729

In [14]:
#conv kernel size = 7 (instead of 3 in initial model) 
a = Input(shape=(54, 54, 3))
#conv layer
c1 = Conv2D(64, 7,7, activation='relu', border_mode='same')(a)
c1p = MaxPooling2D(pool_size=(2, 2)) (c1)
c1d = Dropout(0.1)(c1p)

c2 = Conv2D(128, 7,7, activation='relu', border_mode='same')(c1d)
c2p = MaxPooling2D(pool_size=(2, 2)) (c2)
c2d = Dropout(0.1)(c2p)

c3 = Conv2D(192, 7,7, activation='relu', border_mode='same')(c2d)
c3p = MaxPooling2D(pool_size=(2, 2)) (c3)
c3d = Dropout(0.1)(c3p)


c4 = Conv2D(256, 7,7, activation='relu', border_mode='same')(c3d)
c4p = MaxPooling2D(pool_size=(2, 2)) (c4)
c4d = Dropout(0.1)(c4p)


c5 = Conv2D(256, 7,7, activation='relu', border_mode='same')(c4d)
c5p = MaxPooling2D(pool_size=(2, 2)) (c5)
c5d = Dropout(0.1)(c5p)

#linear layer
flat = Flatten()(c5d)
d1 = Dense(128, activation='relu')(flat)
d1d = Dropout(0.1)(d1)

#output
o_length = Dense(7, activation='softmax')(d1d)
o_digit1 = Dense(11, activation='softmax')(d1d)
o_digit2 = Dense(11, activation='softmax')(d1d)
o_digit3 = Dense(11, activation='softmax')(d1d)
o_digit4 = Dense(11, activation='softmax')(d1d)
o_digit5 = Dense(11, activation='softmax')(d1d)
model = Model(a, [o_length,o_digit1,o_digit2,o_digit3,o_digit4,o_digit5])

rms = RMSprop(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=rms)


In [16]:
#training
result = model.fit(images, [labels[:,:7], labels[:,7:18], labels[:,18:29], labels[:,29:40], labels[:,40:51], labels[:,51:62]], 32, 20, validation_split=0.1, shuffle=True)

Train on 135278 samples, validate on 15031 samples
Epoch 1/20
135278/135278 [==============================] - 721s - loss: 3.8159 - dense_16_loss: 0.3698 - dense_17_loss: 1.1673 - dense_18_loss: 1.3314 - dense_19_loss: 0.7576 - dense_20_loss: 0.1686 - dense_21_loss: 0.0212 - val_loss: 1.9464 - val_dense_16_loss: 0.1683 - val_dense_17_loss: 0.5749 - val_dense_18_loss: 0.6433 - val_dense_19_loss: 0.4310 - val_dense_20_loss: 0.1275 - val_dense_21_loss: 0.0015
Epoch 2/20
135278/135278 [==============================] - 731s - loss: 1.8562 - dense_16_loss: 0.1628 - dense_17_loss: 0.5483 - dense_18_loss: 0.6095 - dense_19_loss: 0.4098 - dense_20_loss: 0.1223 - dense_21_loss: 0.0036 - val_loss: 1.3332 - val_dense_16_loss: 0.1192 - val_dense_17_loss: 0.3990 - val_dense_18_loss: 0.4308 - val_dense_19_loss: 0.2861 - val_dense_20_loss: 0.0963 - val_dense_21_loss: 0.0019
Epoch 3/20
135278/135278 [==============================] - 734s - loss: 1.4013 - dense_16_loss: 0.1228 - dense_17_loss: 0.4183

In [17]:
#validation
score = model.predict(test_images)

score = np.concatenate(score, axis=1)
#round score to get actual prediction instead of probability
roundedScore = np.zeros(score.shape, dtype="int32")
#choose threshold
roundedScore[score > 0.5] = 1

#rounded acc (exactly match)
accuracy_score(test_labels, roundedScore)

0.8867732471109514

In [6]:
#conv kernel size = 9 (instead of 3 in initial model) 
a = Input(shape=(54, 54, 3))
#conv layer
c1 = Conv2D(64, 9,9, activation='relu', border_mode='same')(a)
c1p = MaxPooling2D(pool_size=(2, 2)) (c1)
c1d = Dropout(0.1)(c1p)

c2 = Conv2D(128, 9,9, activation='relu', border_mode='same')(c1d)
c2p = MaxPooling2D(pool_size=(2, 2)) (c2)
c2d = Dropout(0.1)(c2p)

c3 = Conv2D(192, 9,9, activation='relu', border_mode='same')(c2d)
c3p = MaxPooling2D(pool_size=(2, 2)) (c3)
c3d = Dropout(0.1)(c3p)


c4 = Conv2D(256, 9,9, activation='relu', border_mode='same')(c3d)
c4p = MaxPooling2D(pool_size=(2, 2)) (c4)
c4d = Dropout(0.1)(c4p)


c5 = Conv2D(256, 9,9, activation='relu', border_mode='same')(c4d)
c5p = MaxPooling2D(pool_size=(2, 2)) (c5)
c5d = Dropout(0.1)(c5p)

#linear layer
flat = Flatten()(c5d)
d1 = Dense(128, activation='relu')(flat)
d1d = Dropout(0.1)(d1)

#output
o_length = Dense(7, activation='softmax')(d1d)
o_digit1 = Dense(11, activation='softmax')(d1d)
o_digit2 = Dense(11, activation='softmax')(d1d)
o_digit3 = Dense(11, activation='softmax')(d1d)
o_digit4 = Dense(11, activation='softmax')(d1d)
o_digit5 = Dense(11, activation='softmax')(d1d)
model = Model(a, [o_length,o_digit1,o_digit2,o_digit3,o_digit4,o_digit5])

rms = RMSprop(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=rms)


In [8]:
#training
result = model.fit(images, [labels[:,:7], labels[:,7:18], labels[:,18:29], labels[:,29:40], labels[:,40:51], labels[:,51:62]], 32, 20, validation_split=0.1, shuffle=True)

Train on 135278 samples, validate on 15031 samples
Epoch 1/20


109408/135278 [=======================>......] - ETA: 169s - loss: 3.9682 - dense_2_loss: 0.3847 - dense_3_loss: 1.2416 - dense_4_loss: 1.3744 - dense_5_loss: 0.7799 - dense_6_loss: 0.1702 - dense_7_loss: 0.0172- ETA: 721s - loss: 5.7714 - dense_2_loss: 0.6468 - dense_3_loss: 1.8507 - dense_4_loss: 1.9922 - dense_5_loss: 1.0160 -  - ETA: 708s - loss: 5.7110 - dense_2_loss: 0.6345 - dense_3_loss: 1.8319 - dense_4_loss: 1.9740 - dense_5_loss: 1.0084 - dense_6_loss: 0.2172 - dens - ETA: 703s - loss: 5.6892 - dense_2_loss: 0.6303 - dense_3_loss: 1.8251 - den - ETA: 678s - loss: 5.5702 - dense_2_loss: 0.6081 - dense_3_loss: 1.7876 - dense_4_loss: 1.9321 - dense_5_loss: 0 - ETA: 663s - loss: 5.5092 - dense_2_loss: 0.5973 - dense_3_loss: 1.7661 - dense_4_loss: 1.9137 - dense_5_loss: 0.9841 - dense_6_loss: 0.2079 - dense_7_los - ETA: 661s - loss: 5.4984 - dense_2_loss: 0.5955 - dense_3_loss: 1.7623 - dense_4_loss: 1.9106 - dense_5_loss: 0.9826 - dense_6_loss: - ETA: 652s - loss: 5.4637 - dense

 36416/135278 [=======>......................] - ETA: 624s - loss: 1.4310 - dense_2_loss: 0.1176 - dense_3_loss: 0.4277 - dense_4_loss: 0.4628 - dense_5_loss: 0.3143 - dense_6_loss: 0.1048 - dense_7_loss: 0.0038    - ETA: 875s - loss: 1.4376 - dense_2_loss: 0.1016 - dense_3_loss: 0.4537 - dense_4_loss: 0.4837 - dense_5_loss: 0.3178 - dense_6_loss: 0.0807 -  - ETA: 868s - loss: 1.4777 - dense_2_loss: 0.1154 - dense_3_loss: 0.4425 - dense_4_loss: 0.4843 - dense_5_loss: 0.3350 - dense_6_loss: 0.1003 - d - ETA: 860s - loss: 1.4394 - dense_2_loss: 0.1080 - dense_3_loss: 0.4257 - dense_4_loss: 0.4653 - dense_5_loss: 0.3379 - dense_6_loss: 0.1025 - dense_7_loss: - ETA: 854s - loss: 1.4439 - dense_2_loss: 0.1089 - dense_3_loss: 0.4246 - dense_4_loss: 0.4665 - dense_5_loss: 0.3349 - dense_6_loss:  - ETA: 840s - loss: 1.4179 - dense_2_loss: 0.1074 - dense_3_loss: 0.4287 - dense_4_loss: 0.4615 - dense_5_loss: 0.3190 - dense_6_loss: 0.1013 - dense_7_loss: 1.0371e- - ETA: 839s - loss: 1.4300 - dens

 71712/135278 [==============>...............] - ETA: 401s - loss: 1.4261 - dense_2_loss: 0.1206 - dense_3_loss: 0.4306 - dense_4_loss: 0.4594 - dense_5_loss: 0.3051 - dense_6_loss: 0.1060 - dense_7_loss: 0.0043- ETA: 615s - loss: 1.4344 - dense_2_loss: 0.1186 - dense_3_loss: 0.4283 - dense_4_loss: 0.4636 - dense_5_loss: 0.3145 - dense_6_loss: 0.1048 - dense_7_lo - ETA: 613s - loss: 1.4340 - dense_2_loss: 0.1186 - dense_3_loss: 0.4288 - dense_4_loss: 0.4632 - dense_5_loss: 0.3143 - dense_6_loss: 0.1047  - ETA: 609s - loss: 1.4346 - dense_2_loss: 0.1183 - dense_3_loss: 0.4289 - dense_4_loss: 0.4638 - dense_5_loss: 0.3144 - dense_6_loss: 0.1050 - d - ETA: 605s - loss: 1.4353 - dense_2_loss: 0.1188 - dense_3_loss: 0.4289 - dense_4_loss: 0.4629 - dense_5_loss: 0.3142 - dense_6_loss: 0.1053 - dense_7_loss:  - ETA: 603s - loss: 1.4338 - dense_2_loss: 0.1185 - dense_3_loss: 0.4285 - dense_4_loss: 0.4626 - dense_5_loss: 0.314 - ETA: 594s - loss: 1.4305 - dense_2_loss: 0.1183 - dense_3_loss: 0.

135278/135278 [==============================] - 887s - loss: 1.3900 - dense_2_loss: 0.1197 - dense_3_loss: 0.4190 - dense_4_loss: 0.4448 - dense_5_loss: 0.2989 - dense_6_loss: 0.1035 - dense_7_loss: 0.0041 - val_loss: 1.1741 - val_dense_2_loss: 0.1206 - val_dense_3_loss: 0.3716 - val_dense_4_loss: 0.3459 - val_dense_5_loss: 0.2486 - val_dense_6_loss: 0.0853 - val_dense_7_loss: 0.0020se_4_loss: 0.4596 - dense_5_loss: 0.3048 - dense_6_loss: 0.1057 - - ETA: 380s - loss: 1.4240 - dense_2_loss: 0.1203 - dense_3_loss: 0.4292 - dense_4_loss: 0.4591 - dense_5_loss: 0.3049 - ETA: 370s - loss: 1.4216 - dense_2_loss: 0.1195 - dense_3_loss: 0.4286 - dense_4_loss: 0.4586 - dense_5_loss: 0.3045 - dense_6_los - ETA: 364s - loss: 1.4207 - dense_2_loss: 0.1195 - dense_3_loss: 0.4285 - dense_4_loss: 0.4580 - dense_5_loss: 0.3044 - dense_6_loss: 0.1059 - - ETA: 360s - loss: 1.4207 - dense_2_loss: 0.1194 - dense_3_loss: 0.4284 - dense_4_loss: 0.4578 - dense_5_loss: 0.3048 - d - ETA: 351s - loss: 1.4198 -

...

135278/135278 [==============================] - 903s - loss: 0.9223 - dense_2_loss: 0.0798 - dense_3_loss: 0.2875 - dense_4_loss: 0.2890 - dense_5_loss: 0.1878 - dense_6_loss: 0.0740 - dense_7_loss: 0.0043 - val_loss: 0.7420 - val_dense_2_loss: 0.0702 - val_dense_3_loss: 0.2360 - val_dense_4_loss: 0.2235 - val_dense_5_loss: 0.1537 - val_dense_6_loss: 0.0566 - val_dense_7_loss: 0.0021oss: 0.2770 - dense_5_loss: 0.1803 - dense_6_loss: 0.0745 - ETA: 604s - loss: 0.8926 - dense_2_loss: 0.0773 - dense - ETA: 580s - loss: 0.8940 - dense_2_loss: 0.0774 - den - ETA: 557s - loss: 0.8960 - dense_2_loss: 0.0773 - dense_3_loss: 0.2800 - dense_4_loss: 0.2784 - dense_5_loss: 0.1809 - dense_6_loss: 0.0740 - dense_7 - ETA: 554s - loss: 0.8967 - dense_2_loss: 0.0774 - dense_3_loss: 0.2798 - dense_4_loss: 0.2790 - dense_5_loss: 0.1810 - dense_6_loss: 0.0740 - den - ETA: 551s - loss: 0.8993 - dense_2_loss: 0.0778 - dense_3_loss: 0.2799 - dense_4_loss: 0.2801 - dense_5_loss: 0.1820 - dense_6_lo - ETA: 54

 76480/135278 [===============>..............] - ETA: 375s - loss: 0.8617 - dense_2_loss: 0.0723 - dense_3_loss: 0.2684 - dense_4_loss: 0.2717 - dense_5_loss: 0.1761 - dense_6_loss: 0.0695 - dense_7_loss: 0.0037    - ETA: 865s - loss: 0.7517 - dense_2_loss: 0.0523 - dense_3_loss: 0.2538 - dense_4_loss: 0.2208 - dense_5_loss: 0.1749 - dense_6_loss: 0.0498 - dense_7_loss: 1.19 - ETA: 858s - loss: 0.7106 - dense_2_loss: 0.0522 - dense_3_loss: 0.2493 - dense_4_loss: 0.2031 - dense_5_loss: 0.1578 - dense_6_loss: 0.0482 - dense_7_loss:  - ETA: 852s - loss: 0.6622 - dense_2_loss: 0.0468 - dense_3_loss: 0.2293 - dense_4_loss: 0.1977 - dense_5_loss: 0.1416 - dense_6_loss: 0.0468 - dens - ETA: 844s - loss: 0.6814 - dense_2_loss: 0.0454 - dense_3_loss: 0.2337 - dense_4_loss: 0.1942 - dense_5_loss: 0.1 - ETA: 834s - loss: 0.7643 - dense_2_loss: 0.0580 - dense_3_loss: 0.2555 - dense_4_loss: 0.2246 - dense_5_loss: 0.1565 - dense - ETA: 828s - loss: 0.7819 - dense_2_loss: 0.0570 - dense_3_loss: 0.253

135278/135278 [==============================] - 894s - loss: 0.7353 - dense_2_loss: 0.0660 - dense_3_loss: 0.2362 - dense_4_loss: 0.2260 - dense_5_loss: 0.1467 - dense_6_loss: 0.0562 - dense_7_loss: 0.0042 - val_loss: 0.7551 - val_dense_2_loss: 0.0829 - val_dense_3_loss: 0.2356 - val_dense_4_loss: 0.2389 - val_dense_5_loss: 0.1456 - val_dense_6_loss: 0.0500 - val_dense_7_loss: 0.0021
Epoch 14/20
135278/135278 [==============================] - 858s - loss: 0.7239 - dense_2_loss: 0.0644 - dense_3_loss: 0.2363 - dense_4_loss: 0.2237 - dense_5_loss: 0.1427 - dense_6_loss: 0.0526 - dense_7_loss: 0.0042 - val_loss: 0.7517 - val_dense_2_loss: 0.0970 - val_dense_3_loss: 0.2317 - val_dense_4_loss: 0.2089 - val_dense_5_loss: 0.1593 - val_dense_6_loss: 0.0527 - val_dense_7_loss: 0.0021
Epoch 15/20


125408/135278 [==========================>...] - ETA: 62s - loss: 0.7152 - dense_2_loss: 0.0616 - dense_3_loss: 0.2321 - dense_4_loss: 0.2231 - dense_5_loss: 0.1413 - dense_6_loss: 0.0526 - dense_7_loss: 0.0045 - ETA: 470s - loss: 0.6878 - dense_2_loss: 0.0574 - dense_3_loss: 0.2297 - dense_4_loss: 0.2099 - dense_5_loss: 0 - ETA: 459s - loss: 0.6914 - dense_2_loss: 0.0584 - dense_3_loss: 0.2304 - ETA: 377s - loss: 0.6935 - dense_2_loss: 0.0576 - dense_3_loss: 0.2321 - dense_4_loss: 0.2103 -  - ETA: 363s - loss: 0.6947 - dens - ETA: 337s - loss: 0.6940 - dense_2_loss: 0.0576 - dense_3_loss: 0.2313 - dense_4_loss: - ETA: 322s - loss: 0.6947 - dense_2_loss: 0.0578 - dense_3_loss: 0.2313 - dense_4_loss: 0.2115 - dense_5_loss: 0.1367 - dense_6_loss: 0.0523 - dense_7_loss: 0.0 - ETA: 321s - loss: 0.6946 - dense_2_loss: 0.0578 - dense_3_loss: 0.2313 - dense_4_loss: 0.2114 - dense_5_loss: 0.1366 - dense_6_loss: 0 - ETA: 315s - loss: 0.7043 - dense_2_loss: 0.0601 - dense_3_loss: 0.2338 - dense_

135278/135278 [==============================] - 870s - loss: 0.7030 - dense_2_loss: 0.0615 - dense_3_loss: 0.2289 - dense_4_loss: 0.2158 - dense_5_loss: 0.1418 - dense_6_loss: 0.0507 - dense_7_loss: 0.0042 - val_loss: 0.5964 - val_dense_2_loss: 0.0768 - val_dense_3_loss: 0.1790 - val_dense_4_loss: 0.1827 - val_dense_5_loss: 0.1169 - val_dense_6_loss: 0.0389 - val_dense_7_loss: 0.0021: 0.7854 - dense_2_loss: 0.0794 - dense_3_loss: 0.2897 - dense_4_loss: 0.2183 - dense_5_loss: 0.1768 - dense_6_loss: 0.0211 - dense_7_loss: 1.2086e - ETA: 838s - loss: 0.7497 - dense_2_loss: 0.0751 - dense_3_loss: 0.2798 - dense_4_loss: 0.2089 - dense_5_loss: 0.1657 - dense_6_loss: 0.0201 - dense_7_loss - ETA: 836s - loss: 0.7473 - dense_2_loss: 0.0798 - dense_3_loss: 0.2436 - dense_4_loss: 0.2398 - dense_5_loss: 0.1491 - dense_6_loss: 0.0349 - dense_7_loss: 1.2025e - ETA: 836s - loss: 0.7342 - dense_2_loss: 0.0745 - dense_3_loss: 0.2420 - dense_4_loss: 0.2354 - dense_5_loss: 0.1446 - dense_6_loss: 0.0377 

In [9]:
#validation
score = model.predict(test_images)

score = np.concatenate(score, axis=1)
#round score to get actual prediction instead of probability
roundedScore = np.zeros(score.shape, dtype="int32")
#choose threshold
roundedScore[score > 0.5] = 1

#rounded acc (exactly match)
accuracy_score(test_labels, roundedScore)

0.85623615352374105